In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


In [8]:
# Load the dataset with the correct column name for image paths (e.g., 'path')
data = pd.read_csv('HAM10000_metadata.csv')


In [9]:
# Preprocess the images and labels
images = []
labels = []
image_directory = 'C:\\Users\\rafih\\PycharmProjects\\ML\\Skin Cancer'  # Replace with your absolute image directory path
for index, row in data.iterrows():
    image_path = os.path.join(image_directory, row['image_id'] + '.jpg')
    image = cv2.imread(image_path)
    if image is not None:
        image = cv2.resize(image, (224, 224))
        images.append(image)
        labels.append(row['dx'])

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize the image data
images = images / 255.0

In [10]:
meta_data = pd.read_csv("HAM10000_metadata.csv")
meta_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [11]:
print("Unique Cancer types represented in data.")
print(meta_data.dx.unique(),"\n")

# Handling categorical data
encoder = LabelEncoder()
meta_data["dx_label"] = encoder.fit_transform(meta_data["dx"])

# Display of labels and their integer encoding
print("Cancer types and their integer encoding")
print(encoder.classes_)
print(encoder.transform(encoder.classes_))

Unique Cancer types represented in data.
['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec'] 

Cancer types and their integer encoding
['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']
[0 1 2 3 4 5 6]


In [12]:


# Convert labels to numerical classes
unique_labels = np.unique(labels)
label_to_index = {label: index for index, label in enumerate(unique_labels)}
labels = np.array([label_to_index[label] for label in labels])

# Convert labels to one-hot encoded vectors
num_classes = len(unique_labels)
labels = to_categorical(labels, num_classes)

In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)



In [14]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [15]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation for better generalization
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [ ]:
# Train the model with data augmentation
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=15, validation_data=(X_test, y_test))


Epoch 1/15
251/251 [==============================] - 370s 1s/step - loss: 1.1102 - accuracy: 0.6630 - val_loss: 0.9607 - val_accuracy: 0.6680
Epoch 2/15
251/251 [==============================] - 330s 1s/step - loss: 0.9732 - accuracy: 0.6659 - val_loss: 0.8819 - val_accuracy: 0.6665
Epoch 3/15
251/251 [==============================] - 325s 1s/step - loss: 0.9053 - accuracy: 0.6716 - val_loss: 0.9499 - val_accuracy: 0.6695
Epoch 4/15
251/251 [==============================] - 322s 1s/step - loss: 0.8927 - accuracy: 0.6755 - val_loss: 0.8904 - val_accuracy: 0.6610
Epoch 5/15
251/251 [==============================] - 319s 1s/step - loss: 0.8673 - accuracy: 0.6767 - val_loss: 0.8376 - val_accuracy: 0.6835
Epoch 6/15
251/251 [==============================] - 308s 1s/step - loss: 0.8668 - accuracy: 0.6833 - val_loss: 0.8143 - val_accuracy: 0.6760
Epoch 7/15
251/251 [==============================] - 308s 1s/step - loss: 0.8246 - accuracy: 0.6865 - val_loss: 0.8833 - val_accuracy: 0.6785

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)


In [ ]:
# Extracting metrics
metrics = history.history

train_loss = metrics["loss"]
train_accuracy = metrics["accuracy"]

test_loss = metrics["val_loss"]
test_accuracy = metrics["val_accuracy"]

# Visualizing metrics
plt.figure(figsize = (13,4))
plt.subplot(1,2,1)
plt.title("Loss.")
plt.plot(train_loss, label = "Train");
plt.plot(test_loss, label = "Test");
plt.grid(True)
plt.legend(loc = "best");

plt.subplot(1,2,2)
plt.title("Accuracy.")
plt.plot(train_accuracy, label = "Train");
plt.plot(test_accuracy, label = "Test");
plt.grid(True)
plt.legend(loc = "best");